<a href="https://colab.research.google.com/github/TavionK/CPSC-419/blob/main/projects/Bach/Bach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bach Project
The goal of this notebook is to predict if the chord is a major or minor cord based on the notes it contains.

1) Import necessary libraries

In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


2) Read in the data and see what all we have

In [93]:
bach = pd.read_csv('https://raw.githubusercontent.com/TavionK/CPSC-419/main/projects/Bach/bach.csv')
#bach.set_index('choral_ID', inplace=True)
#bach = bach.drop('event_number', axis=1)
bach

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


2) Format the data

I have noticed that some of the data (the chords and bass) have data that won't work so I have to change the yes and no's to true and false and one hot encode the bass column.

In [94]:
bach['c'] = bach['C'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('C', axis=1)
bach['c#'] = bach['C#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('C#', axis=1)
bach['d'] = bach['D'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('D', axis=1)
bach['d#'] = bach['D#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('D#', axis=1)
bach['e'] = bach['E'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('E', axis=1)
bach['f'] = bach['F'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('F', axis=1)
bach['f#'] = bach['F#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('F#', axis=1)
bach['g'] = bach['G'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('G', axis=1)
bach['g#'] = bach['G#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('G#', axis=1)
bach['a'] = bach['A'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('A', axis=1)
bach['a#'] = bach['A#'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('A#', axis=1)
bach['b'] = bach['B'].apply(lambda x: True if x == 'YES' else False)
bach = bach.drop('B', axis=1)
#bach = bach.join(one_hot)
bach

,choral_ID,event_number,bass,meter,chord_label,c,c#,d,d#,e,f,f#,g,g#,a,a#,b
0,000106b_,1,F,3,F_M,True,False,False,False,False,True,False,False,False,True,False,False
1,000106b_,2,E,5,C_M,True,False,False,False,True,False,False,True,False,False,False,False
2,000106b_,3,E,2,C_M,True,False,False,False,True,False,False,True,False,False,False,False
3,000106b_,4,F,3,F_M,True,False,False,False,False,True,False,False,False,True,False,False
4,000106b_,5,F,2,F_M,True,False,False,False,False,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,G,4,G_m,False,False,True,False,False,False,False,True,False,False,True,False
5661,015505b_,106,G,3,G_m,False,False,True,False,False,False,False,True,False,True,False,False
5662,015505b_,107,C,5,C_M,True,False,False,False,True,False,False,True,False,False,False,False
5663,015505b_,108,C,3,C_M,True,False,False,False,True,False,False,True,False,False,True,False


In [95]:
one_hot = pd.get_dummies(bach['bass'])
bach = bach.drop('bass', axis=1)
bach = bach.join(one_hot)
bach

,choral_ID,event_number,meter,chord_label,c,c#,d,d#,e,f,f#,g,g#,a,a#,b,A,A#,Ab,B,Bb,C,C#,D,D#,Db,E,Eb,F,F#,G,G#
0,000106b_,1,3,F_M,True,False,False,False,False,True,False,False,False,True,False,False,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,000106b_,2,5,C_M,True,False,False,False,True,False,False,True,False,False,False,False,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,000106b_,3,2,C_M,True,False,False,False,True,False,False,True,False,False,False,False,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,000106b_,4,3,F_M,True,False,False,False,False,True,False,False,False,True,False,False,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,000106b_,5,2,F_M,True,False,False,False,False,True,False,False,False,True,False,False,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,4,G_m,False,False,True,False,False,False,False,True,False,False,True,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5661,015505b_,106,3,G_m,False,False,True,False,False,False,False,True,False,True,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5662,015505b_,107,5,C_M,True,False,False,False,True,False,False,True,False,False,False,False,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5663,015505b_,108,3,C_M,True,False,False,False,True,False,False,True,False,False,True,False,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [96]:
bach.set_index('choral_ID', inplace=True)

bach

,event_number,meter,chord_label,c,c#,d,d#,e,f,f#,g,g#,a,a#,b,A,A#,Ab,B,Bb,C,C#,D,D#,Db,E,Eb,F,F#,G,G#
choral_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000106b_,1,3,F_M,True,False,False,False,False,True,False,False,False,True,False,False,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,2,5,C_M,True,False,False,False,True,False,False,True,False,False,False,False,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,3,2,C_M,True,False,False,False,True,False,False,True,False,False,False,False,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
000106b_,4,3,F_M,True,False,False,False,False,True,False,False,False,True,False,False,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
000106b_,5,2,F_M,True,False,False,False,False,True,False,False,False,True,False,False,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
015505b_,105,4,G_m,False,False,True,False,False,False,False,True,False,False,True,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,106,3,G_m,False,False,True,False,False,False,False,True,False,True,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
015505b_,107,5,C_M,True,False,False,False,True,False,False,True,False,False,False,False,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


3) Test data set

Create a simple clf and train and test the data.

In [140]:
bach_train, bach_test = train_test_split(bach, test_size= 0.2)
bach_train_features = bach_train.drop('chord_label', axis=1)
bach_train_labels = bach_train['chord_label']

clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(bach_train_features, bach_train_labels)
bach_test_features = bach_test.drop('chord_label', axis=1)
bach_test_labels = bach_test['chord_label']
predictions = clf.predict(bach_test_features)
accuracy_score(bach_test_labels,predictions)

0.6963812886142984

Try with a max depth of 4

In [144]:
clf4 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)
clf4.fit(bach_train_features, bach_train_labels)
predictions = clf4.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.5463371579876434

4) Bonus

In [145]:
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']
from sklearn.model_selection import train_test_split
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bFeatures, bLabels, test_size = 0.2, random_state=42)

Test the accuray score with a plain clf

In [147]:
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(bach_train_features, bach_train_labels)
predictions = clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.6725507502206531

Test with a cross value of 10 to get an average

In [100]:
scores = cross_val_score(clf, bach_test_features,bach_test_labels, cv=10)
print(scores)
print(scores.mean())

[0.6754386  0.55263158 0.68421053 0.66371681 0.61061947 0.69026549
 0.57522124 0.61946903 0.69911504 0.61061947]
0.6381307250426953


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Test with different max depths

In [148]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)
clf.fit(bach_train_features, bach_train_labels)
predictions = clf.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)


0.5375110326566638

User hyperparameter to find best max depth and min sample split

In [146]:
hyperparam_grid = [
    {'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
     'min_samples_split': [2,3,4, 5]}
  ]
clf = tree.DecisionTreeClassifier(criterion='entropy')
grid_search = GridSearchCV(clf, hyperparam_grid, cv=10)
grid_search.fit(bach_train_features, bach_train_labels)
grid_search.best_params_

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


{'max_depth': 9, 'min_samples_split': 5}

In [109]:
predictions = grid_search.best_estimator_.predict(bach_test_features)
accuracy_score(bach_test_labels, predictions)

0.6990291262135923